In [42]:
import numpy as np
from scipy.optimize import differential_evolution
all_data = np.array(list(map(int, "3"*30 + "012012102210212012010201021020101201202010202102210210201201202101201201201202012210")))

In [43]:
from numba import jit

In [44]:
@jit
def estimate(all_data, predrange, dist_decay, endweight, mult1, mult2):
    match_weight = np.linspace(endweight, 1)
    final_move, final_val = 0, 0
    for movemult, predmove in ((1.0, 0), (mult1, 1), (mult2, 2)):
        totalval = 0
        for loc in np.where(all_data == predmove)[0]:
            match = all_data[loc-predrange: loc] == all_data[-predrange:]
            thisval = movemult * (dist_decay ** loc) * np.sum(match_weight[match])
            totalval += thisval
        if totalval > final_val:
            final_move = predmove
            final_val = totalval
    return final_move

def parameter_estimate(params, all_data):
    dist_decay, endweight, mult1, mult2 = params
    predrange = int(10)
    return estimate(all_data, predrange, dist_decay, endweight, mult1, mult2)

def metric(params, all_data):
    good = 0
    bad = 0
    for i in range(1, 60):
        pred = parameter_estimate(params, all_data[:-i])
        if  pred == all_data[-i]:
            good += 1
        else:
            bad += 1
    return - good / (good + bad)

In [53]:
differential_evolution(metric, [(0.5, 1.0), (0.0, 1.0), (0.0, 2.0), (0.0, 2.0)], tol=0.05, polish=False, args = (all_data,), x0= [0.9, 0.5, 1.0, 1.0]).x

array([0.74686628, 0.14186128, 0.71715571, 0.2113299 ])

In [46]:
[3.882e+00,9.384e-01,8.835e-01,9.050e-01,7.737e-01]

[3.882, 0.9384, 0.8835, 0.905, 0.7737]